In [13]:
import json
import numpy as np
import pickle
# import pandas as pd
# from tqdm  import tqdm

import json, bz2, os
import sys

from tqdm import tqdm
# from ase_write import write_xyz
import pandas as pd
import numpy as np
from pymatgen.core.structure import Structure
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')
from m3gnet.models import M3GNet, Potential
# from sklearn.model_selection import train_test_split
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.structure import Structure
import pickle
import os
from pymatgen.io.vasp.outputs import Vasprun

In [14]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 100})

### Get data from Alloys

In [15]:
print(os.getcwd())
wd = os.getcwd()
data_path = "data_prep_alloys/CE_data_TM_sulphide/"
os.chdir("/local/scratch/fjager/")

/local/scratch/fjager/data_prep_alloys/CE_data_TM_sulphide


In [16]:
def prepare_data(path):
    os.chdir(path)

    run_0 = []
    run_final = []

    for dir in os.listdir('./'):
        if os.path.isdir(dir):
            for dir2 in os.listdir(dir):
                if os.path.isdir(os.path.join(dir,dir2)):
                    for dir3 in os.listdir(os.path.join(dir,dir2)):
                        if os.path.isdir(os.path.join(dir,dir2,dir3)):
                            for dir4 in os.listdir(os.path.join(dir,dir2,dir3)):
                                if os.path.isdir(os.path.join(dir,dir2,dir3,dir4)):
                                    for dir5 in os.listdir(os.path.join(dir,dir2,dir3,dir4)):
                                        if os.path.isdir(os.path.join(dir,dir2,dir3,dir4,dir5)):
                                            for dir6 in os.listdir(os.path.join(dir,dir2,dir3,dir4,dir5)):
                                                for dir7 in os.listdir(os.path.join(dir,dir2,dir3,dir4,dir5,dir6)):
                                                    if dir7 == 'run.0':
                                                        run_0.append(os.path.join(dir,dir2,dir3,dir4,dir5,dir6,dir7))
                                                    elif dir7 == 'run.final':
                                                        run_final.append(os.path.join(dir,dir2,dir3,dir4,dir5,dir6,dir7))
    energies = {}
    forces = {}
    stresses = {}
    structures = {}
    for dir in run_0:
        if os.path.isfile(os.path.join(dir,'vasprun.xml')):
            vasp_run = Vasprun(os.path.join(dir,'vasprun.xml'))
            energies[dir]= [el['e_fr_energy'] for el in vasp_run.ionic_steps]
            forces[dir] = [el['forces'] for el in vasp_run.ionic_steps]
            stresses[dir] = [el['stress'] for el in vasp_run.ionic_steps]
            structures[dir] = [el['structure'] for el in vasp_run.ionic_steps]
    
    os.chdir("/local/scratch/fjager/")
            
    return energies, forces, stresses, structures

In [17]:
%%capture
energies, forces, stresses, structures = prepare_data(data_path)

### Load the relaxed structures

In [4]:
relaxed_structures_path = 'relaxation_alloys/save_model/00971-0.024993-0.002652-0.020979-0.013617relax_struc_0.1_1000_val.pickle'

# Load the pickle file
with open(relaxed_structures_path, 'rb') as file:
    relaxed_structures = pickle.load(file)

In [7]:
print(relaxed_structures.keys())

dict_keys(['CE_Mo-Ta_S2/p_MoS2/training_data/SCEL7_7_1_1_0_4_2/9/calctype.default/run.0', 'CE_Nb-Mo_S2/p_MoS2/training_data/SCEL7_1_7_1_0_0_0/0/calctype.default/run.0', 'CE_Ir-Ru_S2/p-WTe2/training_data/SCEL4_4_1_1_0_0_0/27/calctype.default/run.0', 'CE_Ti-V_S2/p_MoS2/training_data/SCEL4_4_1_1_0_0_2/0/calctype.default/run.0', 'CE_Ir-Ru_S2/p-NbTe2/training_data/SCEL4_4_1_1_0_1_0/22/calctype.default/run.0', 'CE_Ir-Ru_S2/p-NbTe2/training_data/SCEL4_2_2_1_0_0_0/25/calctype.default/run.0', 'CE_Ti-Ta_S2/p_CdI2/training_data/SCEL6_3_2_1_0_2_0/3/calctype.default/run.0', 'CE_Ir-Ru_S2/p-NbTe2/training_data/SCEL4_4_1_1_0_0_1/15/calctype.default/run.0', 'CE_Ti-Ta_S2/p_MoS2/training_data/SCEL6_2_3_1_0_0_1/8/calctype.default/run.0', 'CE_Ti-Nb_S2/p_CdI2/training_data/SCEL3_1_3_1_0_0_0/1/calctype.default/run.0', 'CE_Nb-Mo_S2/p_MoS2/training_data/SCEL7_7_1_1_0_4_2/2/calctype.default/run.0', 'CE_Nb-Mo_S2/p_MoS2/training_data/SCEL7_7_1_1_0_0_6/6/calctype.default/run.0', 'CE_Mo-Ta_S2/p_MoS2/training_data/S

### Relaxation Error

In [71]:
def mae(arr):
    n = len(arr)
    # print(n)
    return sum(arr)/n

def rmse(arr):
    n = len(arr)
    return np.sqrt(sum(np.square(arr))/n)


def area_per_atom(lat_param, nr_atoms):
    area = lat_param[0]*lat_param[1] #yields the Area in Angstrom^2
    res_area_per_atom = area/nr_atoms
    
    return res_area_per_atom

def area_per_atom_sqrt(lat_param, nr_atoms):
    area = lat_param[0]*lat_param[1] #yields the Area in Angstrom^2
    res_area_per_atom = np.sqrt(area)/nr_atoms
    
    return res_area_per_atom

def get_error_alloys(relaxed_structures, train_structures):
    
    runs_compared = 0
    runs_compared_volume = 0
    
    runs_relaxed = list(relaxed_structures.keys()) #this is list of differnet runs
    
    diff_energy_per_atom = []
    diff_area_per_atom = []
    diff_area_per_atom_sqrt = []
    area_per_atom_sqrt_arr = []
    
    for key in runs_relaxed:
        # print("key: {}".format(key))
        for idx, dictionary in enumerate(train_structures):
            if key in dictionary:
                train_run = dictionary
                final_structure_training = train_structures[key][-1]
                # print(final_structure_training)
                nr_sites = len(final_structure_training)
                
                energy_final = energies[key][-1]
                energy_final = energy_final/nr_sites
                
                # Get data from relaxation
                final_struct_relax = relaxed_structures[key]["final_structure"]
                final_struct_relax_energy =  relaxed_structures[key]['energy']
                
                # print(final_struct_relax)
                
            
                if final_struct_relax != None:
            
                    lat_parm_train = final_structure_training.lattice.abc
                    lat_parm_relax = final_struct_relax.lattice.abc

                    area_per_atom_train_sqrt = area_per_atom_sqrt(lat_parm_train, nr_sites) 
                    area_per_atom_relax_sqrt = area_per_atom_sqrt(lat_parm_relax, nr_sites)
                    
                    area_per_atom_train = area_per_atom(lat_parm_train, nr_sites) 
                    area_per_atom_relax = area_per_atom(lat_parm_relax, nr_sites)
                    
                    area_per_atom_sqrt_arr.append(area_per_atom_relax_sqrt)
                    
                    # print(area_per_atom_train)
                    # print(area_per_atom_relax)
                    
                    diff_area_sqrt = np.abs(area_per_atom_train_sqrt - area_per_atom_relax_sqrt)
                    diff_area_org = np.abs(area_per_atom_train - area_per_atom_relax)
                    
                    diff_area_per_atom_sqrt.append(diff_area_sqrt)
                    diff_area_per_atom.append(diff_area_org)
                    
                    runs_compared_volume += 1
            
                
                if final_struct_relax_energy != None:
                    final_structure_relaxation_energy = final_struct_relax_energy/nr_sites
                    diff_energy_per_atom.append(np.abs(energy_final - final_structure_relaxation_energy))
                    # print(final_structure_training_energy, final_structure_relaxation_energy)
                    runs_compared += 1
                    
    
    
    # # MAX absolute error: 0.2568950681217454 eV/atom
    # Median absolute error: 0.0015535010107422902 eV/atom
   
    print("Runs compared: {}".format(runs_compared))
    # print("Runs compared volume: {}".format(runs_compared_volume))
    print("Converged Percentage: {}%".format(np.round(runs_compared/len(runs_relaxed),3)*100))
    # print("Length diff_energy: {}".format(len(diff_energy_per_atom)))
    print("============================")
    print("Resulting MAE area per atom: {}".format(mae(diff_area_per_atom)))
    print("Resulting MAE area per atom sqrt: {}".format(mae(diff_area_per_atom_sqrt)))
    print("Mean area per atom sqrt: {}".format(np.mean(area_per_atom_sqrt_arr)))
    print("============================")
    print("Resulting MAE energy: {}".format(mae(diff_energy_per_atom)))
    print("Max absolute error: {}".format(np.max(diff_energy_per_atom)))
    # print("Median absolute error: {}".format(np.mean(diff_energy_per_atom)))
    print("Standard deviation error: {}".format(np.std(diff_energy_per_atom)))
    

    return

In [72]:
get_error_alloys(relaxed_structures, structures)

Runs compared: 146
Converged Percentage: 100.0%
Resulting MAE area per atom: 0.012933829426582312
Resulting MAE area per atom sqrt: 0.0007777028871350743
Mean area per atom sqrt: 0.4236159389428398
Resulting MAE energy: 0.009816344645547512
Max absolute error: 0.1468529635205078
Standard deviation error: 0.026326303593187196
